In [0]:
# download LJSpeech dataset
!wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
# decompress
!tar -xvjf LJSpeech-1.1.tar.bz2


In [0]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive.  
# You can access to your Drive files using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
!pip install ffmpeg
!pip install torchaudio

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6085 sha256=43814fcb29e2dbae06971acb5485645875876d30469b7614b521fd4362a69709
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
     |████████████████████████████████| 2.7MB 2.7MB/s 
     |████████████████████████████████| 773.1MB 23kB/s 
ERROR: torchvision 0.4.2+cu100 has requirement torch==1.3.1, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.3.1+cu100
    Uninstalling torch-1.3.1+cu100:
      Successfully uninstalled torch-1.3.1+cu100


In [4]:
# lädt mp3 file und konvertiert es zu wav
!ffmpeg -i "/content/gdrive/My Drive/text2speech/DieRachederSithStarWarsEpisode3_ep7_Full length.mp3" -acodec pcm_s16le -ar 22050 sample.wav

[wav @ 0x55717c37cc00] Filesize 4437127656 invalid for wav, output file will be broken
size= 4333132kB time=13:58:27.56 bitrate= 705.6kbits/s speed= 425x    
video:0kB audio:4333132kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000011%


In [0]:
!mkdir /content/audio

In [6]:
cd /content/audio


/content/audio


In [0]:
# erzeugt mp3 Schnipsel mit 8 sec Dauer
!ffmpeg -i "/content/sample.wav" -f segment -segment_time 8 -c copy out%03d.wav


In [8]:
cd /content

/content


In [9]:
!git clone https://github.com/NVIDIA/waveglow.git


Cloning into 'waveglow'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 176 (delta 0), reused 1 (delta 0), pack-reused 173
Receiving objects: 100% (176/176), 434.05 KiB | 600.00 KiB/s, done.
Resolving deltas: 100% (96/96), done.


In [10]:
cd waveglow

/content/waveglow


In [11]:
!git submodule init
!git submodule update

Submodule 'tacotron2' (http://github.com/NVIDIA/tacotron2) registered for path 'tacotron2'
Cloning into '/content/waveglow/tacotron2'...
Submodule path 'tacotron2': checked out 'fc0cf6a89a47166350b65daa1beaa06979e4cddf'


In [0]:
!pip3 install -r /content/waveglow/requirements.txt

In [13]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./


Writing setup.sh


In [0]:
!sh setup.sh

In [0]:
!mkdir /content/log

In [1]:
%%writefile  /content/waveglow/config.json

{
    "train_config": {
        "output_directory": "checkpoints",
        "epochs": 100000,
        "learning_rate": 3e-5,
        "sigma": 1.0,
        "iters_per_checkpoint": 200,
        "batch_size": 3,
        "seed": 1234,
        "checkpoint_path": "/content/gdrive/My Drive/text2speech/ceckpoint_out.pth"
    },
    "data_config": {
        "training_files":"/content/audio",
        "segment_length": 16000,
        "sampling_rate": 22050,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024,
        "mel_fmin": 0.0,
        "mel_fmax": 8000.0
    },
    "dist_config": {
        "dist_backend": "nccl",
        "dist_url": "tcp://localhost:54321"
    },

    "waveglow_config": {
        "n_mel_channels": 80,
        "n_flows": 12,
        "n_group": 8,
        "n_early_every": 4,
        "n_early_size": 2,
        "WN_config": {
            "n_layers": 8,
            "n_channels": 512,
            "kernel_size": 3
        }
    }
}


Overwriting /content/waveglow/config.json


In [2]:
cd /content/waveglow

/content/waveglow


In [0]:
import json
import os
import torch
from convert_model import update_model

#=====START: ADDED FOR DISTRIBUTED======
from distributed import init_distributed, apply_gradient_allreduce, reduce_tensor
from torch.utils.data.distributed import DistributedSampler
#=====END:   ADDED FOR DISTRIBUTED======

from torch.utils.data import DataLoader
from glow import WaveGlow,WaveGlowLoss
from mel2samp import Mel2Samp


out_Path = '/content/gdrive/My Drive/text2speech/ceckpoint_out.pth'
checkpoint_path = '/content/gdrive/My Drive/text2speech/ceckpoint_out.pth'
load_check = 1
from_scratch = 1 - load_check

def load_checkpoint(checkpoint_path, model, optimizer):
    assert os.path.isfile(checkpoint_path)
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    iteration = checkpoint_dict['iteration']
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
    model_for_loading = checkpoint_dict['model']
    model.load_state_dict(model_for_loading.state_dict())
    print("Loaded checkpoint '{}' (iteration {})" .format(
          checkpoint_path, iteration))
    return model, optimizer, iteration

def save_checkpoint(model, optimizer, learning_rate, iteration, filepath):
    print("Saving model and optimizer state at iteration {} to {}".format(
          iteration, filepath))
    #model_for_saving = WaveGlow(**waveglow_config).cuda()
    #model_for_saving.load_state_dict(model.state_dict())
    torch.save({'model': model,
                'iteration': iteration,
                'optimizer': optimizer.state_dict(),
                'learning_rate': learning_rate}, filepath)

def train(num_gpus, rank, group_name, output_directory, epochs, learning_rate,
          sigma, iters_per_checkpoint, batch_size, seed, checkpoint_path):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #=====START: ADDED FOR DISTRIBUTED======
    if num_gpus > 1:
        init_distributed(rank, num_gpus, group_name, **dist_config)
    #=====END:   ADDED FOR DISTRIBUTED======

    criterion = WaveGlowLoss(sigma)
    
    
    model = WaveGlow(**waveglow_config).cuda()
    if from_scratch:
        model = torch.load('/content/gdrive/My Drive/text2speech/waveglow_256channels.pt', map_location='cpu')['model']
    #model = update_model(model)
    model.cuda()
   

    #=====START: ADDED FOR DISTRIBUTED======
    if num_gpus > 1:
        model = apply_gradient_allreduce(model)
    #=====END:   ADDED FOR DISTRIBUTED======

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Load checkpoint if one exists
    iteration = 0
    if load_check:
        model, optimizer, iteration = load_checkpoint(checkpoint_path, model,
                                                      optimizer)
        iteration += 1  # next iteration is iteration + 1

    trainset = Mel2Samp(**data_config)
    # =====START: ADDED FOR DISTRIBUTED======
    train_sampler = DistributedSampler(trainset) if num_gpus > 1 else None
    # =====END:   ADDED FOR DISTRIBUTED======
    train_loader = DataLoader(trainset, num_workers=1, shuffle=False,
                              sampler=train_sampler,
                              batch_size=batch_size,
                              pin_memory=False,
                              drop_last=True)

    # Get shared output_directory ready
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        print("output directory", output_directory)

    model.train()
    epoch_offset = max(0, int(iteration / len(train_loader)))
    # ================ MAIN TRAINNIG LOOP! ===================
    
    steps = 3
    for epoch in range(epoch_offset, epochs):
        print("Epoch: {}".format(epoch))
        for i, batch in enumerate(train_loader):
            model.zero_grad()

            mel, audio = batch
            mel = torch.autograd.Variable(mel.cuda())
            audio = torch.autograd.Variable(audio.cuda())
            outputs = model((mel, audio))

            loss = criterion(outputs)/steps
            loss.backward()
            
            
            if (i+1)%steps == 0:
                print("{}:\t{:.9f}".format(iteration, loss))
                
                optimizer.step()
                model.zero_grad()
               
               
                
            ###
            

       
            if (iteration % iters_per_checkpoint == 0):
                if rank == 0:
                    checkpoint_path = "{}/waveglow_{}".format(
                        output_directory, iteration)
                    save_checkpoint(model, optimizer, learning_rate, iteration,out_Path)

            iteration += 1
            
            
    return model

if __name__ == "__main__":


    # Parse configs.  Globals nicer in this case
    with open('/content/waveglow/config.json') as f:
        data = f.read()
    config = json.loads(data)
    train_config = config["train_config"]
    global data_config
    data_config = config["data_config"]
    global dist_config
    dist_config = config["dist_config"]
    global waveglow_config
    waveglow_config = config["waveglow_config"]



    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = False
    model = train(1, 0,"", **train_config)
